## Week 12 Exercise 12.2 Author: Rex Gayas Course & Section: DSC360-T301 Data Mining: Text Analytics an (2243-1) Date: 02 MAR 2024

##### Loading the Dataset

In [58]:
import pandas as pd

# Define the file path
file_path = "D:\\ALPHA\\Dynamic Folder\\Bellevue\\Winter 2023\\Data Mining\\Week 5\\twitter_sample.csv"

# Load the dataset into a Pandas DataFrame
tweets_df = pd.read_csv(file_path)

# Display the first few rows of the dataframe to verify
print(tweets_df.head())


                Tweet Id                                          Tweet URL  \
0  "1167429261210218497"  https://twitter.com/animalhealthEU/status/1167...   
1  "1167375334670557185"  https://twitter.com/PennyBrohnUK/status/116737...   
2  "1167237977615097861"  https://twitter.com/lordbyronaf/status/1167237...   
3  "1167236897078480898"  https://twitter.com/CountessDavis/status/11672...   
4  "1167228378191204353"  https://twitter.com/Local12/status/11672283781...   

  Tweet Posted Time (UTC)                                      Tweet Content  \
0    30 Aug 2019 13:30:00  Pets change our lives &amp; become a part of o...   
1    30 Aug 2019 09:55:43  Another spot of our #morethanmedicine bus in #...   
2    30 Aug 2019 00:49:54  What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩...   
3    30 Aug 2019 00:45:37  What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩...   
4    30 Aug 2019 00:11:46  What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩...   

  Tweet Type                Client  Retweets

##### Preparing the spaCy Pipeline

In [59]:
import spacy

# Load the English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")


spaCy’s small English  language model has been loaded which will be used to process and tokenize the tweets. It creates a Doc object, which is a sequence of tokens that spaCy uses for all its language processing tasks.

##### Refining Matcher Patterns

In [60]:
import spacy
from spacy.matcher import Matcher

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize the Matcher with the spaCy vocabulary
matcher = Matcher(nlp.vocab)

# Define patterns for animal welfare
animal_welfare_patterns = [
    [{"LOWER": "sick"}, {"POS": "NOUN", "OP": "*"}],  
    [{"LOWER": "animals"}, {"POS": "VERB", "OP": "*"}],  
    [{"LOWER": "health"}, {"POS": "ADJ", "OP": "*"}], 
    [{"LOWER": "nutrition"}, {"POS": "ADJ", "OP": "*"}], 
]

# Add patterns to the matcher
matcher.add("ANIMAL_WELFARE", animal_welfare_patterns)

# Example usage of the matcher on a sample document
doc = nlp("Many animals are sick and require health support for better nutrition.")

# Apply the matcher to the doc
matches = matcher(doc, as_spans=True)

# Extract and display the matched text
for span in matches:
    print(span.text)  


animals
sick
health
nutrition


The matcher is correctly identifying the key terms related to animal welfare within the sample text. This suggests that the patterns are well-defined for these particular terms. Applied the matcher across the dataset to extract mentions of animal welfare from the tweets.

##### Extracting Animal Welfare Mentions from Dataset

In [61]:
import pandas as pd
import spacy
from spacy.matcher import Matcher

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Define the file path 
file_path = "D:\\ALPHA\\Dynamic Folder\\Bellevue\\Winter 2023\\Data Mining\\Week 5\\twitter_sample.csv"

# Load the dataset into a Pandas DataFrame
tweets_df = pd.read_csv(file_path)

# Initialize the Matcher with the spaCy vocabulary
matcher = Matcher(nlp.vocab)

# Define patterns for animal welfare
animal_welfare_patterns = [
    [{"LOWER": "sick"}, {"POS": "NOUN", "OP": "*"}],
    [{"LOWER": "animals"}, {"POS": "VERB", "OP": "*"}],
    [{"LOWER": "health"}, {"POS": "ADJ", "OP": "*"}],
    [{"LOWER": "nutrition"}, {"POS": "ADJ", "OP": "*"}],
]

# Add patterns to the matcher
matcher.add("ANIMAL_WELFARE", animal_welfare_patterns)

# Function to process tweets and extract matches
def extract_animal_welfare(tweet_content):
    doc = nlp(tweet_content)
    matches = matcher(doc, as_spans=True)
    return [span.text for span in matches if span.text.strip() != ""]

# Apply the function to the 'Tweet Content' column
tweets_df['Matched Text'] = tweets_df['Tweet Content'].apply(extract_animal_welfare)

# Filter out rows with no matches
tweets_with_matches = tweets_df[tweets_df['Matched Text'].str.len() > 0]

# Create a new DataFrame with the relevant columns
matches_df = tweets_with_matches[['Name', 'Matched Text']].copy()

# Add the 'Identifier' column with the constant value 'ANIMAL_WELFARE'
matches_df['Identifier'] = 'ANIMAL_WELFARE'

# Display the DataFrame with the matched animal welfare phrases
matches_df.head()


,Name,Matched Text,Identifier
28,Cyton Biosciences,[health],ANIMAL_WELFARE
45,ElancoEurope,"[animals, health, animals]",ANIMAL_WELFARE
47,chaojinjae,"[animals, health, animals]",ANIMAL_WELFARE
51,Ibrahima KOUMA,"[animals, health, animals]",ANIMAL_WELFARE
52,AnimalhealthEurope,"[animals, health, animals]",ANIMAL_WELFARE


Created a new DataFrame “matches_df” that only includes tweets with matched text, along with the authors' names and a constant identifier “ANIMAL_WELFARE”.
The match results are being displayed as lists within the “Matched Tex” column. For future consideration, I think the list of matched phrases can be concatenated into a single string. 
